### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 2;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

### Utility functions

In [7]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [8]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [9]:
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        mse = logs.get("mse");
        if(mse < 0.01):
            print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
            self.model.stop_training = True;

In [10]:
def NN_model(dataset, test = None):
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES+1])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,12,43))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    actual = list(series_transformed[WINDOW_SIZE:, 0]);

    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    results, actual, pure_forecast = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_test, MSE_pure,\
        np.array(pure_forecast[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_test, M_pure, forecast = NN_test(ZONE, plot=False);
        print(ZONE, M_train, M_test, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0096 in round 35 and stopped training

70 70 12
Wall time: 6.26 s
1013 7.141952181512968 38.99083624465034 39.5531216450298 [1129.40559357 1127.2222051  1125.27056025 1122.96287845 1120.42721447
 1117.71795191 1114.87749625 1111.52411946 1107.5939557  1103.69207125
 1099.82060817 1096.02767086]

Got an mse at 0.0098 in round 18 and stopped training

70 70 12
Wall time: 5.58 s
1020 7.120269302323555 60.76871861886024 59.95019782753904 [1129.48945558 1128.37093494 1127.60739296 1126.26503418 1124.4899132
 1122.36864015 1119.89153864 1117.13022736 1114.25254747 1111.35216341
 1108.52219358 1105.83254268]

Got an mse at 0.0100 in round 103 and stopped training

70 70 12
Wall time: 7.6 s
1040 5.886243091970776 62.02244810064732 62.53201231586279 [1092.37867297 1079.14659335 1056.74597765 1058.73370304 1061.23003961
 1064.5210885  1068.56329488 1072.47941435 1076.88589938 1080.47393178
 1083.22866485 1085.38463981]

Got an mse at 0.0100 in round 125 and stopped training

70 


Got an mse at 0.0099 in round 157 and stopped training

70 70 12
Wall time: 8.57 s
1845 10.414002511080454 53.47007801260718 52.83807738864329 [1810.52098338 1809.72303433 1807.33715683 1805.82940807 1804.9266957
 1804.28474572 1803.73425798 1803.08697139 1802.17606125 1800.64892364
 1798.45235546 1796.26568607]

Got an mse at 0.0099 in round 100 and stopped training

70 70 12
Wall time: 7.53 s
1850 13.68743890823618 93.63833123317879 101.48032092633544 [1995.05776326 1993.58847526 1992.34873753 1989.30113053 1985.19176302
 1980.36551981 1974.97645519 1969.13414188 1962.92068881 1956.40196332
 1949.63109285 1942.65549485]

Got an mse at 0.0100 in round 142 and stopped training

70 70 12
Wall time: 8.53 s
1851 11.106181304731969 109.90596786013873 112.69563811648304 [1991.54717701 1986.64611986 1979.96142772 1972.31972036 1963.95839869
 1955.3468929  1946.94806874 1939.21364708 1932.52524175 1927.19330837
 1923.77925756 1921.75437478]

Got an mse at 0.0100 in round 123 and stopped trai


Got an mse at 0.0099 in round 115 and stopped training

70 70 12
Wall time: 7.85 s
2124 14.21073821172615 139.09005996174423 141.78548686918114 [2290.01958419 2271.928376   2298.10708376 2300.21722181 2303.88255799
 2306.88606272 2309.98264149 2313.03343731 2315.97640817 2318.18759015
 2319.11280374 2319.94992187]

Got an mse at 0.0099 in round 188 and stopped training

70 70 12
Wall time: 10 s
2125 11.915648844160534 199.43605415154053 202.4058915458006 [2485.3975539  2482.73614428 2475.98744136 2469.92642774 2467.40393757
 2466.26487767 2465.10174606 2463.92048901 2462.74325477 2461.60059642
 2460.51723481 2459.51203953]

Got an mse at 0.0097 in round 105 and stopped training

70 70 12
Wall time: 7.96 s
2126 15.015051797859789 184.38346492660241 186.1055921571357 [2283.68388763 2285.61753774 2308.99090445 2309.14468012 2309.20445544
 2309.52240214 2310.20523666 2311.09341529 2308.74774036 2298.69022524
 2290.66653163 2283.75606075]

Got an mse at 0.0099 in round 229 and stopped trai


Got an mse at 0.0099 in round 130 and stopped training

70 70 12
Wall time: 8.35 s
2170 7.604243830930195 18.08113132239913 18.27358284413164 [1925.86013514 1912.55618717 1901.33066478 1892.19793144 1885.40279532
 1879.36154152 1876.80868073 1874.82396354 1873.12758709 1871.62820728
 1870.30031837 1869.13237964]

Got an mse at 0.0099 in round 123 and stopped training

70 70 12
Wall time: 8.28 s
2171 10.937565569948475 42.759262744564175 43.66992237904324 [2188.90162533 2182.55203685 2177.46283152 2173.21131937 2172.40599415
 2189.65195803 2206.34835844 2210.61740021 2206.8397465  2199.65076672
 2189.96176183 2178.26901647]

Got an mse at 0.0099 in round 97 and stopped training

70 70 12
Wall time: 7.87 s
2176 12.850434288385511 87.60394076380257 86.64157690873843 [2298.77008716 2302.24313784 2297.81488338 2291.83225116 2285.05004368
 2277.65136002 2269.78270094 2261.58193198 2253.18173264 2244.71024889
 2236.28800389 2228.02299248]

Got an mse at 0.0099 in round 64 and stopped trainin


Got an mse at 0.0100 in round 235 and stopped training

70 70 12
Wall time: 10.5 s
2840 9.969282672887337 125.11911243266589 124.21434065321824 [1803.1680129  1801.14349822 1803.7165358  1805.86548307 1807.40206998
 1807.25916132 1806.1457257  1804.35772398 1802.03689748 1799.39381734
 1796.62685487 1793.89033056]

Got an mse at 0.0099 in round 88 and stopped training

70 70 12
Wall time: 7.29 s
2860 8.128063310148754 27.005911393859993 27.64841903542671 [1225.38797749 1226.34224604 1224.73930904 1222.87787191 1220.80899062
 1218.67661413 1216.5441636  1214.35660171 1212.15069844 1209.95560283
 1207.7937123  1205.65862707]

Got an mse at 0.0099 in round 65 and stopped training

70 70 12
Wall time: 6.88 s
2861 7.870760626617522 50.76037332619906 51.74954422446707 [1209.46873626 1210.99897444 1205.62518217 1199.73079324 1192.61140301
 1183.69698787 1172.70987863 1159.96883832 1146.34672079 1132.59993886
 1118.98581876 1108.09362507]

Got an mse at 0.0099 in round 93 and stopped training

...


Got an mse at 0.0097 in round 46 and stopped training

70 70 12
Wall time: 6.23 s
92805 19.645659094594322 115.86098080101684 114.47077869585479 [1978.29544243 1982.93586482 1980.85358981 1976.57514792 1970.23273553
 1961.97400441 1952.10095366 1941.03401118 1929.24923809 1917.19829353
 1905.24085561 1893.61229862]

Got an mse at 0.0095 in round 33 and stopped training

70 70 12
Wall time: 5.8 s
92806 19.346931083166933 149.1191248974326 157.57231894527132 [2190.72951949 2200.21381394 2205.82810563 2209.42200922 2209.99686762
 2207.79458212 2202.86434279 2195.33536937 2185.40242298 2173.38858195
 2159.76509404 2145.11990198]

Got an mse at 0.0098 in round 31 and stopped training

70 70 12
Wall time: 5.77 s
92832 13.124291661789131 67.91850664644899 70.70266541779513 [1961.62201706 1960.65845411 1960.9579969  1960.17032986 1958.29827375
 1954.90264063 1950.03670249 1944.07428945 1937.30690731 1930.05301136
 1922.6050865  1915.19841269]

Got an mse at 0.0094 in round 74 and stopped trai


Got an mse at 0.0096 in round 29 and stopped training

70 70 12
Wall time: 5.74 s
94401 21.94176168135875 72.83726156839826 77.23435167288036 [2843.57027032 2854.37060706 2864.36398678 2861.44130484 2856.14013655
 2848.15426216 2838.28080346 2826.55585293 2812.88771549 2793.89767962
 2760.20825329 2723.76143621]

Got an mse at 0.0097 in round 38 and stopped training

70 70 12
Wall time: 6.11 s
94403 26.462373963910107 87.54816908057863 85.86000532883317 [3142.71568254 3115.15005816 3069.54526022 3078.20430563 3085.19776982
 3089.44259234 3092.49293644 3094.51045569 3095.54578509 3095.8371381
 3097.33150863 3098.12938353]

Got an mse at 0.0099 in round 35 and stopped training

70 70 12
Wall time: 5.71 s
94404 25.51296375614122 131.50323853312173 133.7077909233753 [3190.10372802 3189.93080902 3183.55529703 3175.02234591 3163.87596952
 3153.17805867 3143.81827504 3135.66237202 3128.10049768 3120.4601419
 3112.671377   3107.85171108]

Got an mse at 0.0098 in round 48 and stopped training


70 70 12
Wall time: 6.07 s
95123 18.917724590108424 32.12913883235205 32.53634226758973 [2794.10891125 2793.75536708 2796.31241969 2797.93873078 2798.7774494
 2798.77943981 2797.95493378 2796.31541419 2793.87745429 2790.66078186
 2786.38716865 2781.29225296]

Got an mse at 0.0097 in round 37 and stopped training

70 70 12
Wall time: 5.86 s
95128 13.440975021262696 110.79929184344923 112.6178582692573 [2528.02934267 2527.71761224 2530.15542723 2533.05887066 2536.53438979
 2540.65935388 2545.4346614  2550.71121338 2553.93513563 2555.00668526
 2556.34207437 2557.63622141]

Got an mse at 0.0099 in round 38 and stopped training

70 70 12
Wall time: 6.09 s
95134 20.789601371755914 71.24825054891065 72.21382155490225 [3175.39635352 3177.76571869 3170.75285444 3161.92166291 3152.11452796
 3141.84754415 3131.45537359 3121.22991419 3111.40598381 3102.15272718
 3093.56195062 3085.64737835]

Got an mse at 0.0100 in round 37 and stopped training

70 70 12
Wall time: 6.14 s
95207 18.349154125704924 

70 70 12
Wall time: 7.1 s
98005 14.81202003873646 116.73815004144303 121.30188283374795 [1977.95449483 1975.19819911 1970.69617919 1963.3738088  1953.72212212
 1941.1045285  1926.32908581 1911.28195928 1898.37729296 1888.75834072
 1883.96382073 1876.32141073]

Got an mse at 0.0098 in round 65 and stopped training

70 70 12
Wall time: 6.37 s
98012 15.013576496444513 75.04536007669002 74.55468635906745 [1787.0483373  1787.01917149 1781.13768485 1773.37131815 1763.10695038
 1749.96799847 1734.53172617 1718.28709148 1701.42009645 1686.66055026
 1670.90481961 1651.98662908]

Got an mse at 0.0099 in round 76 and stopped training

70 70 12
Wall time: 6.64 s
98029 15.108371210956678 75.7944052154512 75.19098123871281 [2145.80040024 2149.88981968 2143.38615201 2137.66109925 2131.46636848
 2124.54761875 2117.66033265 2110.67550114 2103.58679577 2096.71670734
 2090.33762477 2085.22233356]

Got an mse at 0.0099 in round 48 and stopped training

70 70 12
Wall time: 6.09 s
98034 15.09580236732452 10

70 70 12
Wall time: 6.02 s
99207 12.225076442206761 102.65627153923604 104.95913637667269 [993.7742468  995.26149406 995.50811459 994.93206616 993.49481187
 991.10572952 987.69642935 983.24704731 977.81975342 971.58271625
 964.80146841 958.77242117]

Got an mse at 0.0100 in round 154 and stopped training

70 70 12
Wall time: 8.18 s
99501 4.435103080929904 38.63455026991569 40.09931509701708 [1270.82387674 1274.23992063 1275.3508778  1276.26405205 1276.93631034
 1277.43380031 1277.7582737  1277.92132728 1277.9391663  1277.01904476
 1275.51694463 1273.65834159]

Got an mse at 0.0099 in round 225 and stopped training

70 70 12
Wall time: 9.54 s
99504 5.281035972801306 21.93549595495605 21.586521516265705 [1427.75219054 1427.80537563 1430.52640259 1433.68705802 1437.13701695
 1440.69645521 1444.1752468  1447.42383723 1450.33512362 1452.84024818
 1454.90472503 1456.52575658]

Got an mse at 0.0100 in round 202 and stopped training

70 70 12
Wall time: 9.11 s
99508 3.495283515145337 11.468161

In [14]:
print(len(score_dict["RMSE_pure"]), sum(score_dict["RMSE_test"])/len(score_dict["RMSE_test"]),
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

1301 62.75661844737773 63.87474715395805


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_test,RMSE_pure,forecast
0,1013,7.141952,38.990836,39.553122,"[1129.405593565533, 1127.2222050988057, 1125.2..."
1,1020,7.120269,60.768719,59.950198,"[1129.4894555781843, 1128.3709349409223, 1127...."
2,1040,5.886243,62.022448,62.532012,"[1092.3786729740864, 1079.1465933487082, 1056...."
3,1085,7.511315,38.001189,38.672657,"[1168.5224161351161, 1167.9283919673392, 1167...."
4,1104,7.087048,25.731798,25.961284,"[1116.9599759512137, 1115.4425062438265, 1104...."


In [16]:
test.to_csv('NN_feature_window_2.csv', index = False)